In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
URL = "https://palette.tatacliq.com/search/result?searchCategory=all&text=Chambor%20Matte%20Riot%20Lipstick:relevance:inStockFlag:true:&isSuggested=false&isKeywordRedirectEnabled=true&source=search"

In [3]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

In [4]:
webpage = requests.get(URL, headers=HEADERS)

In [5]:
type(webpage.content)

bytes

In [6]:
webpage

<Response [200]>

In [7]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [8]:
soup

<!DOCTYPE html>
<html lang="en-IN"><head><meta charset="utf-8"/><title>Buy Cosmetics Products &amp; Beauty Products Online in India at Best Price | Tata Beauty</title><meta content="Buy cosmetics &amp; beauty products online from Tata Beauty, the online shopping beauty store. Browse makeup, health products &amp; more from top beauty brands. ✔ free shipping* ✔ Cash on Delivery" name="description"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no, maximum-scale=1.0, user-scalable=0" name="viewport"/><link href="/assets/images/favicon.ico" rel="apple-touch-icon"/><link href="/assets/images/favicon.ico" rel="shortcut icon"/><meta content="Tata Beauty" name="application-name"/><meta content="Tata Beauty" name="msapplication-tooltip"/><meta content="/assets/images/favicon.ico" name="thumbnail"/><link href="" rel="canonical"/><link href="" rel="alternate"/><meta content="Tata Beauty" property="og:site_name"/><meta content="Buy Lipstick Online In India At Best Prices | Tata

In [9]:
import sqlite3
# Extract data
product_names = [product.get_text(strip=True) for product in soup.find_all('h2', class_ ='teaser-expanded_shared__h1MAl teaser-expanded_subtitle-plp__rC2Qq')]
actual_prices = [price.get_text(strip=True) for price in soup.find_all('span', class_='teaser-expanded_shared__h1MAl teaser-expanded_acprice__VGL5L')]
discount_prices = [newprice.get_text(strip=True) for newprice in soup.find_all('span', class_='teaser-expanded_shared__h1MAl teaser-expanded_ofprice__wET0Q')]
discount_percentages = [discount.get_text(strip=True) for discount in soup.find_all('span', class_='teaser-expanded_shared__h1MAl teaser-expanded_discount___7nAN')]

# Connect to SQLite database
conn = sqlite3.connect('lipstick.db')
cursor = conn.cursor()

# Create the 'lipstick' table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Tata (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT,
    actual_price TEXT,
    discount_price TEXT,
    discount_percentage TEXT
)
''')

# Insert data into the 'lipstick' table
for name, actual_price, discount_price, discount_percentage in zip(product_names, actual_prices, discount_prices, discount_percentages):
    cursor.execute('''
    INSERT INTO tata (product_name, actual_price, discount_price, discount_percentage)
    VALUES (?, ?, ?, ?)
    ''', (name, actual_price, discount_price, discount_percentage))

# Commit the changes and close the connection
conn.commit()
conn.close()
